In [121]:
import torch as th
import torch

def dd(Angvec, SKdd):

    x,y,z = Angvec
    x2, y2, z2 = x**2, y**2, z**2
    xy, yz, zx = x*y, y*z, z*x
    s3 = 3**0.5
    d = th.stack([s3*xy, s3*yz, 0.5*(3*z2-1), s3*zx, 0.5*s3*(x2-y2)]).reshape(5,1)
    dd0 = d @ d.T
    dd2 = 1/3 * dd0
    dd2 = dd2 + torch.tensor([
        [z2,-zx,2/s3*xy,-yz,0],
        [-zx,x2, -s3/3*yz,-xy,yz],
        [2/s3*xy,-s3/3*yz,2/3-z2,-s3/3*zx,s3/3*(x2-y2)],
        [-yz,-xy,-s3/3*zx,y2,-zx],
        [0,yz,s3/3*(x2-y2),-zx,z2]
    ])
    rot_mat = th.stack([dd0, th.eye(5)-dd0-dd2, dd2], dim=-1)

    hs = th.matmul(rot_mat, SKdd.view(-1))

    return hs

def dd3(Angvec, SKdd):

    x = Angvec[0].item()
    y = Angvec[1].item()
    z = Angvec[2].item()
    s3 = th.sqrt(th.scalar_tensor(3.0)).item()
    rot_mat = th.zeros([5, 5, 3])

    rot_mat[0, 0, :] = th.tensor([-3.0 * x ** 2 * (-1.0 + z ** 2 + x ** 2),
                                    (4.0 * x ** 2 * z ** 2 - z ** 2 + 4.0 * x ** 4 - 4.0 * x ** 2 + 1.0),
                                    (-x ** 2 * z ** 2 + z ** 2 + x ** 2 - x ** 4)])

    rot_mat[1, 0, :] = th.tensor([-3.0 * x * (-1.0 + z ** 2 + x ** 2) * z,
                                    (4.0 * z ** 2 + 4.0 * x ** 2 - 3.0) * z * x,
                                    -x * (z ** 2 + x ** 2) * z])

    rot_mat[2, 0, :] = th.tensor([x * y * s3 * (3.0 * z ** 2 - 1.0) / 2.0,
                                    -2.0 * s3 * y * x * (z ** 2),
                                    x * y * (z ** 2 + 1.0) * s3 / 2.0])

    rot_mat[3, 0, :] = th.tensor([3.0 * (x ** 2) * y * z,
                                    -(4.0 * x ** 2 - 1.0) * z * y,
                                    y * (-1.0 + x ** 2) * z])

    rot_mat[4, 0, :] = th.tensor([3.0 / 2.0 * y * x * (2.0 * x ** 2 - 1.0 + z ** 2),
                                    -2.0 * y * x * (2.0 * x ** 2 - 1.0 + z ** 2),
                                    y * x * (2.0 * x ** 2 - 1.0 + z ** 2) / 2.0])

    rot_mat[0, 1, :] = rot_mat[1, 0, :]

    rot_mat[1, 1, :] = th.tensor([-3.0 * (-1.0 + z ** 2 + x ** 2) * z ** 2,
                                    (4.0 * z ** 4 - 4.0 * z ** 2 + 4.0 * x ** 2 * z ** 2 + 1.0 - x ** 2),
                                    -(-1.0 + z) * (z ** 3 + z ** 2 + x ** 2 * z + x ** 2)])

    rot_mat[2, 1, :] = th.tensor([y * s3 * z * (3.0 * z ** 2 - 1.0) / 2.0,
                                    -z * s3 * (2.0 * z ** 2 - 1.0) * y,
                                    (-1.0 + z ** 2) * s3 * z * y / 2.0])

    rot_mat[3, 1, :] = th.tensor([3.0 * y * x * (z ** 2),
                                    -(4.0 * z ** 2 - 1.0) * y * x,
                                    x * y * (-1.0 + z ** 2)])

    rot_mat[4, 1, :] = th.tensor([3.0 / 2.0 * y * (2.0 * x ** 2 - 1.0 + z ** 2) * z,
                                    -(2.0 * z ** 2 - 1.0 + 4.0 * x ** 2) * z * y,
                                    y * (z ** 2 + 2.0 * x ** 2 + 1.0) * z / 2.0])

    rot_mat[0, 2, :] = rot_mat[2, 0, :]
    rot_mat[1, 2, :] = rot_mat[2, 1, :]

    rot_mat[2, 2, :] = th.tensor([((3.0 * z ** 2 - 1.0) ** 2) / 4.0,
                                    -(3.0 * (-1.0 + z ** 2) * z ** 2),
                                    3.0 / 4.0 * ((-1.0 + z ** 2) ** 2)])

    rot_mat[3, 2, :] = th.tensor([x * (3.0 * z ** 2 - 1.0) * s3 * z / 2.0,
                                    -(2.0 * z ** 2 - 1.0) * x * z * s3,
                                    z * x * s3 * (-1.0 + z ** 2) / 2.0])

    rot_mat[4, 2, :] = th.tensor([(2.0 * x ** 2 - 1.0 + z ** 2) * (3.0 * z ** 2 - 1.0) * s3 / 4.0,
                                    -(2.0 * x ** 2 - 1.0 + z ** 2) * (z ** 2) * s3,
                                    s3 * (2.0 * x ** 2 - 1.0 + z ** 2) * (z ** 2 + 1.0) / 4.0])

    rot_mat[0, 3, :] = rot_mat[3, 0, :]
    rot_mat[1, 3, :] = rot_mat[3, 1, :]
    rot_mat[2, 3, :] = rot_mat[3, 2, :]

    rot_mat[3, 3, :] = th.tensor([3.0 * x ** 2 * z ** 2,
                                    (-4.0 * x ** 2 * z ** 2 + z ** 2 + x ** 2),
                                    (-1.0 + z) * (-z + x ** 2 * z - 1.0 + x ** 2)])

    rot_mat[4, 3, :] = th.tensor([3.0 / 2.0 * x * (2.0 * x ** 2 - 1.0 + z ** 2) * z,
                                    -((2.0 * z ** 2 - 3.0 + 4.0 * x ** 2) * z * x),
                                    (x * (z ** 2 - 3.0 + 2.0 * x ** 2) * z) / 2.0])

    rot_mat[0, 4, :] = rot_mat[4, 0, :]
    rot_mat[1, 4, :] = rot_mat[4, 1, :]
    rot_mat[2, 4, :] = rot_mat[4, 2, :]
    rot_mat[3, 4, :] = rot_mat[4, 3, :]
    rot_mat[4, 4, :] = th.tensor([3.0 / 4.0 * ((2.0 * x ** 2 - 1.0 + z ** 2) ** 2),
                                    -z ** 4 + z ** 2 - 4.0 * x ** 2 * z ** 2 - 4.0 * x ** 4 + 4.0 * x ** 2,
                                    (z ** 4 / 4.0 + x ** 2 * z ** 2 + z ** 2 / 2.0 + 1.0 / 4.0 - x ** 2 + x ** 4)])

    # [5,5,3] dot (3,1) => [5,5,1] => [5,5]
    SKdd_rsp = th.reshape(SKdd, [3, 1])
    hs = th.matmul(rot_mat, SKdd_rsp)
    hs = th.reshape(hs, [5, 5])

    return hs


mat = th.tensor([[
    [0,0,0,0,0],
    [0,1,0,0,0],
    [0,0,0,0,3**0.5/3],
    [0,0,0,0,0],
    [0,0,3**0.5/3,0,0]
],
[
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,0,0,-3**2/3],
    [0,0,0,1,0],
    [0,0,-3**2/3,0,0]
],
[
    [1,0,0,0,0],
    [0,0,0,0,0],
    [0,0,-1,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0.5]
],
[
    [0,0,2/(3**0.5),0,0],
    [0,0,0,-1,0],
    [2/(3**0.5),0,0,0,0],
    [0,-1,0,0,0],
    [0,0,0,0,0]
],
[
    [0,0,0,-1,0],
    [0,0,-3**0.5/3,0,1],
    [0,-3**0.5/3,0,0,0],
    [-1,0,0,0,0],
    [0,1,0,0,0]
],
[
    [0,-1,0,0,0],
    [-1,0,0,0,0],
    [0,0,0,-3**0.5/3,0],
    [0,0,-3**0.5/3,0,-1],
    [0,0,0,-1,0]
],
[
    [0,0,0,0,0],
    [0,0,0,0,1],
    [0,0,2/3,0,0],
    [0,0,0,0,0],
    [0,1,0,0,-0.25]
]
])

def dd1(Angvec, SKdd, mat=mat):

    x,y,z = Angvec
    x2, y2, z2 = x**2, y**2, z**2
    xy, yz, zx = x*y, y*z, z*x
    s3 = 3**0.5
    d = th.stack([s3*xy, s3*yz, 0.5*(3*z2-1), s3*zx, 0.5*s3*(x2-y2)]).reshape(5,1)
    dd0 = d @ d.T
    
    dd2 = 1/3 * dd0 + mat.permute(1,2,0) @ torch.stack([x2,y2,z2,xy,yz,zx,torch.scalar_tensor(1.)])
    rot_mat = th.stack([dd0, th.eye(5)-dd0-dd2, dd2], dim=-1)

    hs = th.matmul(rot_mat, SKdd.view(-1))

    return hs


# def pp2(Angvec, SKpp):

#     x = Angvec[0].item()
#     y = Angvec[1].item()
#     z = Angvec[2].item()
#     rot_mat = th.zeros([3, 3, 2])
#     # [3,3,2] dot (2,1) => [3,3,1] => [3,3]
#     rot_mat[0, 0, :] = th.tensor([1.0 - z ** 2 - x ** 2, z ** 2 + x ** 2])
#     rot_mat[0, 1, :] = th.tensor([z * y, - z * y])
#     rot_mat[0, 2, :] = th.tensor([x * y, - x * y])
#     rot_mat[1, 0, :] = rot_mat[0, 1, :]
#     rot_mat[1, 1, :] = th.tensor([z ** 2, 1.0 - z ** 2])
#     rot_mat[1, 2, :] = th.tensor([z * x, - z * x])
#     rot_mat[2, 0, :] = rot_mat[0, 2, :]
#     rot_mat[2, 1, :] = rot_mat[1, 2, :]
#     rot_mat[2, 2, :] = th.tensor([x ** 2, 1 - x ** 2])

#     return rot_mat

#     SKpp_rsp = th.reshape(SKpp, [2, 1])
#     rot_mat.requires_grad = False
#     hs = th.matmul(rot_mat, SKpp_rsp)
#     hs = th.reshape(hs, [3, 3])

#     return hs


fn1 = torch.jit.trace(dd, [torch.tensor([0., 0., 1.]), torch.tensor([-2.7, 3.5,-2.5])])
# fn2 = torch.jit.script(pd1, [torch.tensor([0., 0., 1.]), torch.tensor([-2.7, 3.5])])

/tmp/ipykernel_10171/2207207660.py:6: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  x,y,z = Angvec
/tmp/ipykernel_10171/2207207660.py:13: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  dd2 = dd2 + torch.tensor([
/tmp/ipykernel_10171/2207207660.py:13: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  dd2 = dd2 + torch.tensor([


In [109]:
ang = torch.tensor([0.3, 0.4, 0.5]) * (2**0.5)
sksd = torch.tensor([-2.7, -3.1,-2.5])
for i in range(50000):
    dd(ang, sksd)

In [23]:
b = torch.randn(7)
for i in range(50000):
    fn2(b)

In [7]:
r2 = 2**0.5
ang = [0.3*r2, 0.4*r2, 0.5*r2]
sksd = [-2.7, -3.1]
for i in range(50000):
    pd2(ang, sksd)

AttributeError: 'float' object has no attribute 'item'

In [118]:
ang = torch.tensor([0.3, 0.4, 0.5]) * (2**0.5)
sksd = torch.tensor([-2.7, -3.1,-2.5])
for i in range(50000):
    fn1(ang, sksd)

In [151]:
ang = torch.tensor([0.3, 0.4, 0.5]) * (2**0.5)

sksd = torch.tensor([-2.7, -3.1])
for i in range(50000):
    fn2(ang, sksd)

In [124]:
result = dd3(ang, sksd)- dd(ang, sksd)
print(dd(ang, sksd)[:,:])
print(result[:,:])

tensor([[-2.6963, -0.0072,  0.2286, -0.1104, -0.0302],
        [-0.0072, -2.7040, -0.0346,  0.0720,  0.1896],
        [ 0.2286, -0.0346, -2.9625, -0.0260, -0.0667],
        [-0.1104,  0.0720, -0.0260, -2.7460, -0.2178],
        [-0.0302,  0.1896, -0.0667, -0.2178, -2.7912]])
tensor([[ 0.0000e+00,  0.0000e+00,  5.9605e-08,  0.0000e+00,  1.8626e-08],
        [ 0.0000e+00,  0.0000e+00,  2.9802e-08, -1.4901e-08,  0.0000e+00],
        [ 5.9605e-08,  2.9802e-08, -2.3842e-07,  0.0000e+00, -1.4901e-08],
        [ 0.0000e+00, -1.4901e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.8626e-08,  0.0000e+00, -1.4901e-08,  0.0000e+00,  0.0000e+00]])


In [8]:
import torch
def fn(a,b=torch.eye(5)):
    return a @ b


ffn = torch.jit.trace(fn, [torch.randn(5,5)])

m = torch.randn(5,5)


print(m-ffn(m))

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [3]:
from transform_sk_speed import RotationSK
from transform_se3 import RotationSE3
import torch

sk = RotationSK(rot_type=torch.double, device=torch.device('cpu'))
se3 = RotationSE3(rot_type=torch.double, device=torch.device('cpu'))

# initial rotate H or S func.
# initial rotate H or S func.


In [ ]:
from e3nn.o3 import wigner_3j, Irrep, xyz_to_angles, Irrep
import torch
from typing import Tuple

@torch.jit.script
def transform_o3(Angvec: torch.Tensor, L_vec: torch.Tensor, irs: torch.Tensor, dtype=torch.float64, device="cpu"):
    """_summary_

    Parameters
    ----------
    Angvec : torch.Tensor
        direction cosines of shift vector \hat{R}, in order [y,z,x].
    L_vec : torch.Tensor 
        looks like torch.tensor([l1, l2]), where l1 <= l2.
    irs : torch.Tensor
        the irreducible representation of operator block under basis of sperical harmonics
        denoted by l1 and l2.
    """
    # assert len(irs.shape) in [1,2]
    # assert len(Angvec.shape) in [1,2]
    # assert len(L_vec) == 2
    
    if len(irs.shape) == 1:
        irs = irs.unsqueeze(0)
    if len(Angvec.shape) == 1:
        Angvec = Angvec.unsqueeze(0)

    l1, l2 = L_vec[0], L_vec[1]
    wms = []
    # assert len(irs.reshape(-1)) == (2*l1+1) * (2*l2+1) * len(Angvec)
    for l_ird in torch.arange(abs(l2-l1), l2+l1+1):
        wms.append(wigner_3j(l1.long(), l2.long(), l_ird, dtype=dtype, device=device) * (2*l_ird+1)**0.5)
    wms = torch.cat(wms, dim=-1)
    angle = xyz_to_angles(Angvec) # (tensor(N), tensor(N))
    rot_mat_L = Irrep(int(l1), 1).D_from_angles(angle[0], angle[1], torch.tensor(0., dtype=dtype, device=device)) # tensor(N, 2l1+1, 2l1+1)
    rot_mat_R = Irrep(int(l2), 1).D_from_angles(angle[0], angle[1], torch.tensor(0., dtype=dtype, device=device)) # tensor(N, 2l2+1, 2l2+1)
    
    
    HR = compose_rotate(cg_basis=wms, irs=irs, rot_mat_L=rot_mat_L, rot_mat_R=rot_mat_R)

    return HR

@torch.jit.script
def compose_rotate(cg_basis: torch.Tensor, irs: torch.Tensor, rot_mat_L: torch.Tensor, rot_mat_R: torch.Tensor):
    H_ird = torch.sum(cg_basis[None,:,:,:] * irs[:,None, None, :], dim=-1)
    HR = rot_mat_L @ H_ird @ rot_mat_R.transpose(1,2)
    return HR

RuntimeError: 
Arguments for call are not valid.
The following variants are available:
  
  aten::mul.Tensor(Tensor self, Tensor other) -> Tensor:
  Expected a value of type 'Tensor' for argument 'self' but instead found type 'NoneType'.
  
  aten::mul.Scalar(Tensor self, Scalar other) -> Tensor:
  Expected a value of type 'Tensor' for argument 'self' but instead found type 'NoneType'.
  
  aten::mul.out(Tensor self, Tensor other, *, Tensor(a!) out) -> Tensor(a!):
  Expected a value of type 'Tensor' for argument 'self' but instead found type 'NoneType'.
  
  aten::mul.Scalar_out(Tensor self, Scalar other, *, Tensor(a!) out) -> Tensor(a!):
  Expected a value of type 'Tensor' for argument 'self' but instead found type 'NoneType'.
  
  aten::mul.left_t(t[] l, int n) -> t[]:
  Could not match type NoneType to List[t] in argument 'l': Cannot match List[t] to NoneType.
  
  aten::mul.right_(int n, t[] l) -> t[]:
  Expected a value of type 'int' for argument 'n' but instead found type 'NoneTy

In [40]:
import time

angvec = torch.randn(1000,3)
angvec = angvec / torch.norm(angvec, dim=-1, keepdim=True)
L_vec = (2,2)
irs = torch.randn(1000,(2*L_vec[0]+1)*(2*L_vec[1]+1))

start = time.time()
transform_o3(Angvec=angvec, L_vec=L_vec, irs=irs, dtype=torch.float32)

end = time.time()
print(end-start)


0.025645017623901367
